In [1]:
from ibapi.client import *
from ibapi.wrapper import *
import pandas as pd
import time
import sys
import numpy as np

In [2]:
class TestApp(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []
        self.done = False
        
    def nextValidId(self, orderId: int):
        mycontract = Contract()
        mycontract.symbol = "TSLA"
        mycontract.secType = "STK"
        mycontract.exchange = "SMART"
        mycontract.currency = "USD"
        
        self.reqHistoricalData(orderId, mycontract, "20230131-14:30:00", "6 M", "5 mins", "TRADES", 0, 1, 0, [])
        
    def historicalData(self, reqId, bar):
        self.data.append(bar)
        
    def historicalDataEnd(self, reqId, start, end):
        self.disconnect()
        
        
app = TestApp()
app.connect("127.0.0.1", 7497, 0)                
try:
    app.run()
    #while not app.done:
        #time.sleep(1)
    #df = app.historicalDataEnd(0, '', '')
except KeyboardInterrupt:
    print("Interrupted by user.")
    app.disconnect()
    sys.exit()

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:eufarmnj
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil
ERROR 1 2176 Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 91.23 to 91


In [3]:
data = []
for bar in app.data:
    data.append([
        bar.date,
        bar.open,
        bar.high,
        bar.low,
        bar.close,
        bar.volume,
        bar.average,
        bar.barCount,
    ])

df = pd.DataFrame(data, columns=["Date", "Open", "High", "Low", "Close", "Volume", "Average", "Bar Count"])
print(df.head())


                 Date      Open      High       Low     Close  Volume  \
0  20220805  03:00:00  312.2233  312.2233  310.4333  311.1333      91   
1  20220805  03:05:00  311.1900  311.6767  311.1900  311.6600      22   
2  20220805  03:10:00  311.3333  311.6667  311.3333  311.5000      43   
3  20220805  03:15:00  311.5000  311.5000  311.0000  311.0000      37   
4  20220805  03:20:00  311.1800  311.1800  311.0000  311.0000      19   

      Average  Bar Count  
0  311.167000         20  
1  311.407333          6  
2  311.450667          8  
3  311.227000          9  
4  311.069000          4  


In [4]:
# Set the date as the index of the dataframe
df.set_index(df['Date'], inplace=True)
display(df)

,Date,Open,High,Low,Close,Volume,Average,Bar Count
Date,,,,,,,,
20220805 03:00:00,20220805 03:00:00,312.2233,312.2233,310.4333,311.1333,91,311.167000,20
20220805 03:05:00,20220805 03:05:00,311.1900,311.6767,311.1900,311.6600,22,311.407333,6
20220805 03:10:00,20220805 03:10:00,311.3333,311.6667,311.3333,311.5000,43,311.450667,8
20220805 03:15:00,20220805 03:15:00,311.5000,311.5000,311.0000,311.0000,37,311.227000,9
20220805 03:20:00,20220805 03:20:00,311.1800,311.1800,311.0000,311.0000,19,311.069000,4
...,...,...,...,...,...,...,...,...
20230131 08:05:00,20230131 08:05:00,164.8900,165.0000,164.8200,164.9900,1235,164.977000,393
20230131 08:10:00,20230131 08:10:00,164.9800,165.0000,164.6400,164.7600,597,164.836000,214
20230131 08:15:00,20230131 08:15:00,164.7700,164.8900,164.6300,164.7200,582,164.747000,215


In [5]:
# Calculate MACD
df['macd_fast_ema'] = df['Close'].ewm(span=12).mean()
df['macd_slow_ema'] = df['Close'].ewm(span=26).mean()
df['macd_line'] = df['macd_fast_ema'] - df['macd_slow_ema']
df['macd_signal_line'] = df['macd_line'].ewm(span=9).mean()

# Define Exponential Moving Average (EMA) parameters
fast_window = 12
slow_window = 26

# Define Bollinger Band parameters
window = 20
multiplier = 2

# Calculate moving average and standard deviation
df['ma'] = df['Close'].rolling(window=window).mean()
df['std'] = df['Close'].rolling(window=window).std()

# Calculate upper and lower bands
df['upper_band'] = df['ma'] + multiplier * df['std']
df['lower_band'] = df['ma'] - multiplier * df['std']

# Calculate EMAs
df['fast_ema'] = df['Close'].ewm(span=fast_window).mean()
df['slow_ema'] = df['Close'].ewm(span=slow_window).mean()

# Define Relative Strength Index (RSI) parameters
rsi_window = 14

# Calculate RSI
delta = df['Close'].diff()
Gain = delta.where(delta > 0, 0)
Loss = -delta.where(delta < 0, 0)
avg_Gain = Gain.rolling(window=rsi_window).mean()
avg_Loss = Loss.rolling(window=rsi_window).mean()
rs = avg_Gain / avg_Loss
df['rsi'] = 100 - (100 / (1 + rs))

# Define stop-loss and take-profit parameters
stop_loss_ratio = 0.03
take_profit_ratio = 0.05

# Set starting balance and risk amount
starting_balance = 1000000
risk_amount = (.02 * starting_balance) # 2% of starting balance
balance = starting_balance

# Calculate order amount
order_amount = risk_amount / stop_loss_ratio

# Initialize the position, profit, trades, wins, and total_pl
position = 0
profit = 0
trades = 0
wins = 0
total_pl = 0

# Iterate over the data
for i, row in df.iterrows():
    # Check if RSI is above 50
    if row['rsi'] > 50:
        # Check if close price is above upper Bollinger Band
        if row['Close'] > row['upper_band']:
            # Check if there is no open position
            if position == 0:
                # Enter a long position
                position = 1
                entry_price = row['Close']
                stop_loss_price = entry_price * (1 - stop_loss_ratio)
                take_profit_price = entry_price * (1 + take_profit_ratio)
    # Check if RSI is below 50
    
    # Check if MACD fast moving average crosses above slow moving average and RSI is below 50
    elif df['fast_ema'][i] > df['slow_ema'][i] and row['rsi'] < 50:
        # Check if there is no open position
        if position == 0:
            # Enter a long position
            position = 1
            entry_price = row['Close']
            # Keep track of entry date
            entry_date = row['Date']
    # Check if stop-loss is triggered
    elif position == 1 and row['Close'] < stop_loss_price:
        # Exit the long position with a stop-loss
        position = 0
        exit_price = stop_loss_price
        pnl = (exit_price - entry_price) / entry_price
        profit += pnl
        trades += 1
        total_pl += pnl
        if pnl > 0:
            wins += 1
    # Check if take-profit is triggered
    elif position == 1 and row['Close'] > take_profit_price:
        # Exit the long position with a take-profit
        position = 0
        exit_price = take_profit_price
        pnl = (exit_price - entry_price) / entry_price
        profit += pnl
        trades += 1
        total_pl += pnl
        if pnl > 0:
            wins += 1
    # Check if MACD fast moving average crosses below slow moving average or RSI is above 50
    elif df['fast_ema'][i] < df['slow_ema'][i] or row['rsi'] > 50:
        # Check if there is an open position
        if position == 1:
            # Exit the long position
            position = 0
            exit_price = row['Close']
            # Keep track of exit date
            exit_date = row['Date']
            # Calculate profit
            pnl = (exit_price - entry_price) / entry_price
            # Check if profit meets the minimum risk reward ratio
            if pnl >= 0.33 * stop_loss_ratio:
                # Update profit
                profit += pnl
                trades += 1
                total_pl += pnl
                if pnl > 0:
                    wins += 1
            else:
                # Re-enter the long position
                position = 1
                entry_price = row['Close']

# Print results
total_balance = balance * profit
win_rate = wins / trades
print("Original Balance: ${:.2f}".format(starting_balance))
print("Win Rate: {:.2f}%".format(win_rate * 100))
print("Number of Trades:", trades)
print("Profit/Loss Ratio: {:.2f}".format(profit))
print("Total Balance after Trades: ${:.2f}".format(total_balance))

Original Balance: $1000000.00
Win Rate: 58.04%
Number of Trades: 143
Profit/Loss Ratio: 1.34
Total Balance after Trades: $1338883.81
